In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
import tensorflow._api.v2.compat.v1 as tf


import algorithms.heartrate as hr
import utils

2023-03-06 00:16:43.250713: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# tensorflow settings
tf.logging.set_verbosity(tf.logging.ERROR)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

# disable warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# x_data_train, y_data_train, groups_train = ...
# x_data_test, y_data_test, groups_test = ...

# dummy:
train_size, test_size = 774, 258
n_groups_train, n_groups_test = 27, 9
data = np.genfromtxt('data/experiments/sleeprawlive.csv', delimiter=';')
data = data[:len(data)-len(data)%400]
data = data.reshape( int(len(data)/400), 400)
x_data_train = np.expand_dims(data, axis=2)
data = np.genfromtxt('data/experiments/gold.csv', delimiter=',')
data = data[:len(data)-len(data)%16]
y_data_train = np.mean(data.reshape(int(len(data)/16), 16), axis=1)
y_data_train = y_data_train[:774]
groups_train = 27

x_data_test = x_data_train[200:458]
y_data_test = y_data_train[200:458]
groups_test = 9

In [4]:
modelnames = [
    "models.deeper_fcn-x1",
    #"models.stacked_cnn_rnn_improved-x1",
]

In [5]:
predictions_dl_train = {"true": y_data_train, "groups": groups_train}
predictions_dl_test = {"true": y_data_test, "groups": groups_test}

In [6]:
def get_modelpath(modelname):
    return os.path.join("output", modelname)  # modify this if necessary

def get_predictions(modelname, weights_format="weights-{:02d}.h5",
                    batch_size=32, train=False):
    modelpath = get_modelpath(modelname)
    model = utils.get_model_from_json(modelpath)
    
    if not train:
        model.load_weights(os.path.join(modelpath,
                                        weights_format.format(0)))
        y_pred = model.predict(x_data_test)
        tf.keras.backend.clear_session()
        return y_pred[:, 0]

    splitter = utils.get_group_splitter(n_groups_train, groups_train)
    results = []
    for i, (_, v_inds) in enumerate(splitter):
        model.load_weights(os.path.join(modelpath, weights_format.format(i)))
        y_pred = model.predict(x_data_train[v_inds], batch_size=batch_size)
        results = np.r_[results, y_pred[:, 0]]  # append new predictions
    tf.keras.backend.clear_session()
    return results

In [7]:
utils.get_model_from_json(get_modelpath(modelnames[-1])).summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 400, 1)]          0         
                                                                 
 conv1d_132 (Conv1D)         (None, 390, 8)            96        
                                                                 
 batch_normalization_96 (Bat  (None, 390, 8)           32        
 chNormalization)                                                
                                                                 
 leaky_re_lu_120 (LeakyReLU)  (None, 390, 8)           0         
                                                                 
 conv1d_133 (Conv1D)         (None, 190, 8)            712       
                                                                 
 batch_normalization_97 (Bat  (None, 190, 8)           32        
 chNormalization)                                         

In [8]:
# calculate MAE and MAPE patient-wise (for training and testing set)
dl_results_train = np.zeros((len(modelnames), 1, 2, n_groups_train))
dl_results_test = np.zeros((len(modelnames), 1, 2, n_groups_test))

for res, preds_ in [(dl_results_train, predictions_dl_train),
                    (dl_results_test, predictions_dl_test)]:
    y_true, preds = preds_["true"], preds_
    print(preds)
    for ni, name in enumerate(modelnames):
        print(name)
        y_pred = preds[name]
        for gi, group in enumerate(np.unique(preds["groups"])):
            yt = y_true[preds["groups"] == group]
            yp = y_pred[preds["groups"] == group]
            res[ni, 0, :, gi] = (hr.hr_mape(yt, yp), hr.hr_mae(yt, yp))

score_index = 1  # 0: MAPE, 1: MAE
rows = []
for ni, modelname in enumerate(modelnames):
    errmean = dl_results_test.mean(axis=-1)[ni, 0, score_index]
    errstd = dl_results_test.std(axis=-1)[ni, 0, score_index]
    rows.append((modelname, errmean, errstd))

print("Results on testing data:")
print("========================\n")
print(tabulate(rows, tablefmt="presto", floatfmt=".3f",
               headers=["modelname", "mae" if score_index==1 else "mape", "std"]))

{'true': array([28.4375, 17.5625, 46.25  , 49.9375, 26.5625, 18.875 , 30.6875,
       63.9375, 32.5625, 23.5   , 20.1875, 78.375 , 56.625 , 35.375 ,
       19.8125, 42.3125, 55.875 , 25.5   , 15.0625, 18.375 , 56.4375,
       28.0625, 18.75  , 12.5625, 59.5625, 48.0625, 27.8125, 16.25  ,
       47.9375, 73.5625, 36.25  , 23.25  , 25.9375, 73.5625, 39.375 ,
       28.    , 23.625 , 80.0625, 62.125 , 42.6875, 30.6875, 63.0625,
       74.4375, 46.9375, 35.125 , 42.375 , 79.4375, 43.875 , 29.5625,
       23.    , 65.6875, 42.1875, 28.5   , 19.125 , 61.4375, 57.625 ,
       34.375 , 21.9375, 58.9375, 77.6875, 48.4375, 36.375 , 44.5   ,
       84.8125, 49.6875, 36.75  , 29.9375, 68.8125, 47.5625, 34.0625,
       24.0625, 55.5625, 55.8125, 35.4375, 24.625 , 48.4375, 80.5625,
       46.4375, 33.125 , 32.1875, 80.3125, 49.375 , 35.0625, 25.0625,
       63.375 , 55.25  , 37.625 , 26.125 , 55.3125, 67.5   , 39.375 ,
       25.1875, 50.75  , 82.6875, 51.1875, 38.875 , 41.5   , 85.4375,
       52.5

KeyError: 'models.deeper_fcn-x1'